# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __aci app__ and our __aks app__.

The __aci app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __aks app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

---

### Import package and load .env

In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os
%load_ext dotenv
%dotenv

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

### Add video to storage account
To test things locally, download the video stored in this location `https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4`, and reupload it to the Azure storage account you created. 

1. Download the video using `wget`
2. Upload the downloaded video to your storage container using `azcopy`
3. Remove the downloaded video to clean up your directory

In [ ]:
!wget https://happypathspublic.blob.core.windows.net/videos/orangutan.mp4

In [ ]:
!azcopy \
    --source orangutan.mp4 \
    --destination https://{get_key(env_path, "STORAGE_ACCOUNT_NAME")}.blob.core.windows.net/{get_key(env_path, "STORAGE_CONTAINER_NAME")}/orangutan.mp4 \
    --dest-key {get_key(env_path, "STORAGE_ACCOUNT_KEY")} \
    --resume "."

In [ ]:
!rm orangutan.mp4

### Preprocess video 
The first thing we want to test locally is the preprocessing step. This step will use a reference to the video file in blob and perform the following:
1. Download the video to a tmp location
2. Use `ffmpeg` to extract the audio file from the video and save the file to a tmp location
3. Use `ffmpeg` to split the video up into individual frames and save all frames to the tmp location
4. Upload the extracted audio file & frames to storage to be consumed by the next step

In [ ]:
input_frames_dir = "orangutan_frames_test"
audio_file = "orangutan_audio_test.aac"

Run the preprocess script:
- `--frames-dir` is the directory in Azure blob that will be created to store individual frames from preprocessing the video.
- `--audio` is the audio file name in Azure blob to save the extracted audio clip to.

After running this step, check your storage account to see if your `frames_dir` and `audio_file` is loaded.

In [ ]:
!python aci/preprocess.py \
    --video orangutan.mp4 \
    --frames-dir $input_frames_dir \
    --audio $audio_file

### Test neural style transfer locally

Now that the video is preprocessed into individual frames, test the style transfer script locally:
- Add 10 images to queue
- Dequeue those 10 images and apply neural style transfer locally
- Check that the queue is now empty

In [ ]:
output_frames_dir = "orangutan_frames_local_test_processed"

In [ ]:
!python aci/add_images_to_queue.py \
    --input-dir $input_frames_dir \
    --output-dir $output_frames_dir \
    --style "mosaic" \
    --queue-limit 10

The `--input-dir` is the directory in your blob container that corresponds to the queue messages (the script will look inside the directory specified by `--input-dir` to find and process the message that it dequeued).

In [ ]:
!python aks/main.py --terminate

In [ ]:
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"

---

### Conclusion

Check that our `.env` file looks correct.

In [ ]:
!cat .env

Continue to the next [notebook](/notebooks/03_style_transfer_on_aks.ipynb).